In [3]:
import os
import numpy as np
from skimage import io
from matplotlib import pyplot as plt
from scipy.misc import imresize
from scipy import ndimage

%matplotlib inline

import random
import csv
import re
import pickle
import json
from tqdm import tqdm

DATADIR = os.path.expanduser('~/data/kaggle/ultrasound-nerve-segmentation')
PROCDIR = os.path.expanduser('~/ml/kaggle/ultrasound-nerve-segmentation/processed')
MODELDIR = os.path.expanduser('~/ml/kaggle/ultrasound-nerve-segmentation/models')

In [4]:
with open(os.path.join(PROCDIR, 'train_files_map.pkl'), 'rb') as f:
    train_files_map = pickle.load(f)
with open(os.path.join(PROCDIR, 'test_files_map.pkl'), 'rb') as f:
    test_files_map = pickle.load(f)

## create dataset

In [10]:
orig_img_rows, orig_img_cols = 420, 580
img_rows, img_cols = 128, 128
img_channels = 1

# 95/5 train/val split, by subject
data_images_train = []
data_masks_train = []
data_presence_train = []
data_images_val = []
data_masks_val = []
data_presence_val = []

subjects = set([f['subject'] for f in train_files_map.values()])
subjects_val = np.random.choice(list(subjects), int(len(subjects) * 0.05), replace=False)

for file_info in tqdm(train_files_map.values()):
    subject, img, folder, img_file, mask_file = (file_info['subject'], 
                                                 file_info['img'], 
                                                 file_info['folder'], 
                                                 file_info['img_file'], 
                                                 file_info['mask_file'])
    img_filepath = os.path.join(folder, img_file)
    mask_filepath = os.path.join(folder, mask_file)
    image = io.imread(img_filepath) / 255.0
    mask = io.imread(mask_filepath) / 255.0
    if np.sum(mask) == 0:
        presence = False
    else:
        presence = True
    image_resized = imresize(image[:, (orig_img_cols - orig_img_rows):orig_img_cols], 
                             size=(img_rows, img_cols), interp='bilinear')
    mask_resized = imresize(mask[:, (orig_img_cols - orig_img_rows):orig_img_cols], 
                            size=(img_rows, img_cols), interp='bilinear').astype(np.bool)
    if subject in subjects_val:
        data_images_val.append(image_resized)
        data_masks_val.append(mask_resized)
        data_presence_val.append(presence)
    else:
        data_images_train.append(image_resized)
        data_masks_train.append(mask_resized)
        data_presence_train.append(presence)

data_images_train = np.expand_dims(np.array(data_images_train, dtype=np.float32), axis=3)
data_masks_train = np.expand_dims(np.array(data_masks_train, dtype=np.bool), axis=3)
data_presence_train = np.array(data_presence_train, dtype=np.bool)
data_images_val = np.expand_dims(np.array(data_images_val, dtype=np.float32), axis=3)
data_masks_val = np.expand_dims(np.array(data_masks_val, dtype=np.bool), axis=3)
data_presence_val = np.array(data_presence_val, dtype=np.bool)

print('image shapes:', data_images_train.shape, data_images_val.shape)
print('mask shapes:', data_masks_train.shape, data_masks_val.shape)
print('presence label shapes:', data_presence_train.shape, data_presence_val.shape)

 16%|█▌        | 907/5635 [00:19<01:38, 48.23it/s]/home/leon/install/miniconda3/lib/python3.5/site-packages/skimage/external/tifffile/tifffile.py:1794: RuntimeWarning: py_decodelzw encountered unexpected end of stream
  strip = decompress(strip)
100%|██████████| 5635/5635 [01:59<00:00, 47.02it/s]


image shapes: (5395, 128, 128, 1) (240, 128, 128, 1)
mask shapes: (5395, 128, 128, 1) (240, 128, 128, 1)
presence label shapes: (5395,) (240,)


## save

In [11]:
with open(os.path.join(PROCDIR, 'data_train_val_13.pkl'), 'wb') as f:
    pickle.dump((data_images_train, data_masks_train, data_presence_train, 
                 data_images_val, data_masks_val, data_presence_val), f, protocol=4)